In [8]:
!pip install max==25.4.0 --index-url https://dl.modular.com/public/nightly/python/simple/

Looking in indexes: https://dl.modular.com/public/nightly/python/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.0/285.0 MB 6.7 MB/s eta 0:00:00


In [9]:
!git clone https://github.com/modular/mojo-gpu-puzzles

Cloning into 'mojo-gpu-puzzles'...
remote: Enumerating objects: 6332, done.
remote: Counting objects: 100% (481/481), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 6332 (delta 448), reused 416 (delta 416), pack-reused 5851 (from 3)
Receiving objects: 100% (6332/6332), 148.65 MiB | 29.36 MiB/s, done.
Resolving deltas: 100% (3922/3922), done.


In [10]:
!curl -fsSL https://astral.sh/uv/install.sh | sh

downloading uv 0.8.15 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!


In [11]:
import max.support.notebook

In [12]:
def save_code_to_file(text: str, filename: str):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(text)

In [37]:
mojo_code = """
from gpu import thread_idx, block_idx, block_dim, barrier
from gpu.sync import (
    mbarrier_init,
    mbarrier_arrive,
    mbarrier_test_wait,
    async_copy_arrive,
    cp_async_bulk_commit_group,
    cp_async_bulk_wait_group,
)
from gpu.host import DeviceContext
from gpu.memory import async_copy_wait_all
from layout import Layout, LayoutTensor
from layout.tensor_builder import LayoutTensorBuild as tb
from layout.layout_tensor import copy_dram_to_sram_async
from sys.info import sizeof
from sys import info
from sys.arg import argv
from testing import assert_true, assert_almost_equal

# ANCHOR: multi_stage_pipeline

alias TPB = 256  # Threads per block for pipeline stages
alias SIZE = 1024  # Image size (1D for simplicity)
alias BLOCKS_PER_GRID = (4, 1)
alias THREADS_PER_BLOCK = (TPB, 1)
alias dtype = DType.float32
alias layout = Layout.row_major(SIZE)

# Multi-stage processing configuration
alias STAGE1_THREADS = TPB // 2
alias STAGE2_THREADS = TPB // 2
alias BLUR_RADIUS = 2


fn multi_stage_image_blur_pipeline[
    layout: Layout
](
    output: LayoutTensor[mut=True, dtype, layout],
    input: LayoutTensor[mut=False, dtype, layout],
    size: Int,
):

    # Shared memory buffers for pipeline stages
    input_shared = tb[dtype]().row_major[TPB]().shared().alloc()
    blur_shared = tb[dtype]().row_major[TPB]().shared().alloc()

    global_i = block_dim.x * block_idx.x + thread_idx.x
    local_i = thread_idx.x

    # Stage 1: Load and preprocess (threads 0-127)
    # FILL ME IN (roughly 10 lines)
    if local_i < STAGE1_THREADS:
        if global_i < size:
            input_shared[local_i] = input[global_i] * 1.1
            if local_i + STAGE1_THREADS < size:
                input_shared[local_i + STAGE1_THREADS] = (
                    input[global_i + STAGE1_THREADS] * 1.1
                )
            else:
                input_shared[local_i] = 0.0
                if local_i + STAGE1_THREADS < TPB:
                    input_shared[local_i + STAGE1_THREADS] = 0.0

    barrier()  # Wait for Stage 1 completion

    # Stage 2: Apply blur (threads 128-255)
    # FILL ME IN (roughly 25 lines)
    if local_i >= STAGE1_THREADS:
        blur_idx = local_i - STAGE1_THREADS
        var blur_sum: Scalar[dtype] = 0.0
        blur_count = 0

        for offset in range(-BLUR_RADIUS, BLUR_RADIUS + 1):
            sample_idx = blur_idx + offset
            if sample_idx >= 0 and sample_idx < TPB:
                blur_sum += rebind[Scalar[dtype]](input_shared[sample_idx])
                blur_count += 1

        if blur_count > 0:
            blur_shared[blur_idx] = blur_sum / blur_count
        else:
            blur_shared[blur_idx] = 0.0

        second_idx = blur_idx + STAGE1_THREADS
        if second_idx < TPB:
            blur_sum = 0.0
            blur_count = 0
            for offset in range(-BLUR_RADIUS, BLUR_RADIUS + 1):
                sample_idx = second_idx + offset
                if sample_idx >= 0 and sample_idx < TPB:
                    blur_sum += rebind[Scalar[dtype]](input_shared[sample_idx])
                    blur_count += 1

            if blur_count > 0:
                blur_shared[second_idx] = blur_sum / blur_count
            else:
                blur_shared[second_idx] = 0.0

    barrier()  # Wait for Stage 2 completion

    # Stage 3: Final smoothing (all threads)
    # FILL ME IN (roughly 7 lines)
    if global_i < size:
        final_value = blur_shared[local_i]

        if local_i > 0:
            final_value = (final_value + blur_shared[local_i - 1]) * 0.6
        if local_i < 0:
            final_value = (final_value + blur_shared[local_i + 1]) * 0.6

        output[global_i] = final_value

    barrier()  # Ensure all writes complete


# ANCHOR_END: multi_stage_pipeline

# ANCHOR: double_buffered_stencil

# Double-buffered stencil configuration
alias STENCIL_ITERATIONS = 3
alias BUFFER_COUNT = 2


fn double_buffered_stencil_computation[
    layout: Layout
](
    output: LayoutTensor[mut=True, dtype, layout],
    input: LayoutTensor[mut=False, dtype, layout],
    size: Int,
):

    # Double-buffering: Two shared memory buffers
    buffer_A = tb[dtype]().row_major[TPB]().shared().alloc()
    buffer_B = tb[dtype]().row_major[TPB]().shared().alloc()

    # Memory barriers for coordinating buffer swaps
    init_barrier = tb[DType.uint64]().row_major[1]().shared().alloc()
    iter_barrier = tb[DType.uint64]().row_major[1]().shared().alloc()
    final_barrier = tb[DType.uint64]().row_major[1]().shared().alloc()

    global_i = block_dim.x * block_idx.x + thread_idx.x
    local_i = thread_idx.x

    # Initialize barriers (only thread 0)
    if local_i == 0:
        mbarrier_init(init_barrier.ptr, TPB)
        mbarrier_init(iter_barrier.ptr, TPB)
        mbarrier_init(final_barrier.ptr, TPB)

    # Initialize buffer_A with input data
    # FILL ME IN (roughly 4 lines)
    if local_i < TPB and global_i < size:
        buffer_A[local_i] = input[global_i]
    else:
        buffer_A[local_i] = 0.0

    # Wait for buffer_A initialization
    _ = mbarrier_arrive(init_barrier.ptr)
    _ = mbarrier_test_wait(init_barrier.ptr, TPB)

    # Iterative stencil processing with double-buffering
    @parameter
    for iteration in range(STENCIL_ITERATIONS):

        @parameter
        if iteration % 2 == 0:
            # Even iteration: Read from A, Write to B
            # FILL ME IN (roughly 12 lines)
            if local_i < TPB:
                var stencil_sum: Scalar[dtype] = 0.0
                var stencil_count: Int = 0

                for offset in range(-1, 2):
                    sample_idx = local_i + offset
                    if sample_idx >= 0 and sample_idx < TPB:
                        stencil_sum += rebind[Scalar[dtype]](buffer_A[sample_idx])
                        stencil_count += 1

                if stencil_count > 0:
                    buffer_B[local_i] = stencil_sum /stencil_count
                else:
                    buffer_B[local_i] = buffer_A[local_i]

        else:
            # Odd iteration: Read from B, Write to A
            # FILL ME IN (roughly 12 lines)
            if local_i < TPB:
                var stencil_sum: Scalar[dtype] = 0.0
                var stencil_count: Int = 0

                for offset in range(-1, 2):
                    sample_idx = local_i + offset
                    if sample_idx >= 0 and sample_idx < TPB:
                        stencil_sum += rebind[Scalar[dtype]](buffer_B[sample_idx])
                        stencil_count += 1
                if stencil_count > 0:
                    buffer_A[local_i] = stencil_sum / stencil_count
                else:
                    buffer_A[local_i] = buffer_B[local_i]

        # Memory barrier: wait for all writes before buffer swap
        _ = mbarrier_arrive(iter_barrier.ptr)
        _ = mbarrier_test_wait(iter_barrier.ptr, TPB)

        # Reinitialize barrier for next iteration
        if local_i == 0:
            mbarrier_init(iter_barrier.ptr, TPB)

    # Write final results from active buffer
    if local_i < TPB and global_i < size:

        @parameter
        if STENCIL_ITERATIONS % 2 == 0:
            # Even iterations end in buffer_A
            output[global_i] = buffer_A[local_i]
        else:
            # Odd iterations end in buffer_B
            output[global_i] = buffer_B[local_i]

    # Final barrier
    _ = mbarrier_arrive(final_barrier.ptr)
    _ = mbarrier_test_wait(final_barrier.ptr, TPB)


# ANCHOR_END: double_buffered_stencil


def test_multi_stage_pipeline():
    with DeviceContext() as ctx:
        out = ctx.enqueue_create_buffer[dtype](SIZE).enqueue_fill(0)
        inp = ctx.enqueue_create_buffer[dtype](SIZE).enqueue_fill(0)

        # Initialize input with a simple pattern
        with inp.map_to_host() as inp_host:
            for i in range(SIZE):
                # Create a simple wave pattern for blurring
                inp_host[i] = Float32(i % 10) + Float32(i / 100.0)

        # Create LayoutTensors
        out_tensor = LayoutTensor[mut=True, dtype, layout](out.unsafe_ptr())
        inp_tensor = LayoutTensor[mut=False, dtype, layout](inp.unsafe_ptr())

        ctx.enqueue_function[multi_stage_image_blur_pipeline[layout]](
            out_tensor,
            inp_tensor,
            SIZE,
            grid_dim=BLOCKS_PER_GRID,
            block_dim=THREADS_PER_BLOCK,
        )

        ctx.synchronize()

        # Simple verification - check that output differs from input and values are reasonable
        with out.map_to_host() as out_host, inp.map_to_host() as inp_host:
            print("Multi-stage pipeline blur completed")
            print("Input sample:", inp_host[0], inp_host[1], inp_host[2])
            print("Output sample:", out_host[0], out_host[1], out_host[2])

            # Basic verification - output should be different from input (pipeline processed them)
            assert_true(
                abs(out_host[0] - inp_host[0]) > 0.001,
                "Pipeline should modify values",
            )
            assert_true(
                abs(out_host[1] - inp_host[1]) > 0.001,
                "Pipeline should modify values",
            )
            assert_true(
                abs(out_host[2] - inp_host[2]) > 0.001,
                "Pipeline should modify values",
            )

            # Values should be reasonable (not NaN, not extreme)
            for i in range(10):
                assert_true(
                    out_host[i] >= 0.0, "Output values should be non-negative"
                )
                assert_true(
                    out_host[i] < 1000.0, "Output values should be reasonable"
                )

            print("✅ Multi-stage pipeline coordination test PASSED!")


def test_double_buffered_stencil():
    with DeviceContext() as ctx:
        # Test Puzzle 26B: Double-Buffered Stencil Computation
        out = ctx.enqueue_create_buffer[dtype](SIZE).enqueue_fill(0)
        inp = ctx.enqueue_create_buffer[dtype](SIZE).enqueue_fill(0)

        # Initialize input with a different pattern for stencil testing
        with inp.map_to_host() as inp_host:
            for i in range(SIZE):
                # Create a step pattern that will be smoothed by stencil
                inp_host[i] = Float32(1.0 if i % 20 < 10 else 0.0)

        # Create LayoutTensors for Puzzle 26B
        out_tensor = LayoutTensor[mut=True, dtype, layout](out.unsafe_ptr())
        inp_tensor = LayoutTensor[mut=False, dtype, layout](inp.unsafe_ptr())

        ctx.enqueue_function[double_buffered_stencil_computation[layout]](
            out_tensor,
            inp_tensor,
            SIZE,
            grid_dim=BLOCKS_PER_GRID,
            block_dim=THREADS_PER_BLOCK,
        )

        ctx.synchronize()

        # Simple verification - check that GPU implementation works correctly
        with inp.map_to_host() as inp_host, out.map_to_host() as out_host:
            print("Double-buffered stencil completed")
            print("Input sample:", inp_host[0], inp_host[1], inp_host[2])
            print("GPU output sample:", out_host[0], out_host[1], out_host[2])

            # Basic sanity checks
            var processing_occurred = False
            var all_values_valid = True

            for i in range(SIZE):
                # Check if processing occurred (output should differ from step pattern)
                if abs(out_host[i] - inp_host[i]) > 0.001:
                    processing_occurred = True

                # Check for invalid values (NaN, infinity, or out of reasonable range)
                if out_host[i] < 0.0 or out_host[i] > 1.0:
                    all_values_valid = False
                    break

            # Verify the stencil smoothed the step pattern
            assert_true(
                processing_occurred, "Stencil should modify the input values"
            )
            assert_true(
                all_values_valid,
                "All output values should be in valid range [0,1]",
            )

            # Check that values are smoothed (no sharp transitions)
            var smooth_transitions = True
            for i in range(1, SIZE - 1):
                # Check if transitions are reasonably smooth (not perfect step function)
                var left_diff = abs(out_host[i] - out_host[i - 1])
                var right_diff = abs(out_host[i + 1] - out_host[i])
                # After 3 stencil iterations, sharp 0->1 transitions should be smoothed
                if left_diff > 0.8 or right_diff > 0.8:
                    smooth_transitions = False
                    break

            assert_true(
                smooth_transitions, "Stencil should smooth sharp transitions"
            )

            print("✅ Double-buffered stencil test PASSED!")


def main():
    print("Puzzle 26: GPU Synchronization Primitives")
    print("=" * 50)

    # Parse command line arguments
    if len(argv()) != 2:
        print("Usage: p26.mojo [--multi-stage | --double-buffer]")
        print("  --multi-stage: Test multi-stage pipeline coordination")
        print("  --double-buffer: Test double-buffered stencil computation")
        return

    if argv()[1] == "--multi-stage":
        print("TPB:", TPB)
        print("SIZE:", SIZE)
        print("STAGE1_THREADS:", STAGE1_THREADS)
        print("STAGE2_THREADS:", STAGE2_THREADS)
        print("BLUR_RADIUS:", BLUR_RADIUS)
        print("")
        print("Testing Puzzle 26A: Multi-Stage Pipeline Coordination")
        print("=" * 60)
        test_multi_stage_pipeline()
    elif argv()[1] == "--double-buffer":
        print("TPB:", TPB)
        print("SIZE:", SIZE)
        print("STENCIL_ITERATIONS:", STENCIL_ITERATIONS)
        print("BUFFER_COUNT:", BUFFER_COUNT)
        print("")
        print("Testing Puzzle 26B: Double-Buffered Stencil Computation")
        print("=" * 60)
        test_double_buffered_stencil()
    else:
        print("Usage: p26.mojo [--multi-stage | --double-buffer]")
"""

In [32]:
save_code_to_file(mojo_code, "/content/mojo-gpu-puzzles/problems/p29/p29.mojo")

In [38]:
!cd /content/mojo-gpu-puzzles && uv run poe p29 --double-buffer

Poe => mojo problems/p29/p29.mojo --double-buffer
LLVM ERROR: Cannot select: intrinsic %llvm.nvvm.mbarrier.init.shared
[12399:12399:20250903,184101.535590:ERROR file_io_posix.cc:144] open /sys/devices/system/cpu/cpu0/cpufreq/scaling_cur_freq: No such file or directory (2)
[12399:12399:20250903,184101.535670:ERROR file_io_posix.cc:144] open /sys/devices/system/cpu/cpu0/cpufreq/scaling_max_freq: No such file or directory (2)
Please submit a bug report to https://github.com/modular/modular/issues and include the crash backtrace along with all the relevant source codes.
Stack dump:
0.	Program arguments: /content/mojo-gpu-puzzles/.venv/bin/mojo problems/p29/p29.mojo --double-buffer
1.	Running pass 'Function Pass Manager' on module '<split-module>'.
2.	Running pass 'NVPTX DAG->DAG Pattern Instruction Selection' on function '@p29_double_buffered_stencil_com6A6A_814ca401d45da4e8e6fd48871b2accfc'
Stack dump without symbol names (ensure you have llvm-symbolizer in your PATH or set the environmen